# Lecture et transmission d'une entrée analogique par port série

## Voici le petit programme Arduino :

```c
int analogPinNumber;// Numéro de l'entrée analogique
int analogValue; // Valeur brute de cette entrée (0 à 1024, convertisseur sur 10 bits)

void setup() {
  Serial.begin(9600); // Initialialisation port série 9600 bauds
  analogPinNumber = 0; // Initialialisation numéro de l'entrée analogique
}

void loop() {
  analogValue = analogRead(analogPinNumber); // Acquisition de l'entrée
  Serial.println(analogValue); // Écriture de la valeur acquise sur le port série
  delay(1000); // Attente d'une seconde avant la prochaine lecture
}
```

Il échantillonne toute les secondes l'entrée analogique numéro zéro et transmet la valeur au système hôte par le port série. Vous pouvez par exemple connecter le TMP36 du kit arduino. C'est ce qui est supposé dans la suite pour la partie Python.

---

## Voici la partie Python :

* Création, paramétrage et ouverture du port série
* Lecture et conversion de dix valeurs consécutives avant fermeture du port série

La conversion de la valeur brute reçue de l'Arduino en degrès celsius se fait en deux étapes.

1. Conversion de la valeur brute en volt :

$$Valeur\_Volt = \frac{5}{1024}\cdot Valeur\_brute$$

3. Conversion de la valeur volt en degrès celsius :

$$Valeur\_Celsius = (Valeur\_Volt - 0.75)/0.01 + 25$$

On peut en deduire cette équation d'après le [documentation](https://www.analog.com/media/en/technical-documentation/data-sheets/TMP35_36_37.pdf) du TPM36 puisque sa tension de sortie est de 75mV à 25°C et que sa sensibilité est de 10mV/°C.

In [197]:
# 1. Création, paramétrage et ouverture du port série
import serial
import time

if 'ser' in locals(): ser.close()
ser = serial.Serial()
ser.baudrate = 9600 
ser.port = '/dev/cu.usbmodem14101' # 'COMx' sous windows Ou '/dev/tty.usbmodem1411' par exemple, sous MacOS et Linux
ser.open()
ser.is_open
time.sleep(5)# Attendre que le boot arduino soit terminé

In [198]:
# 2. Lecture et conversion de dix valeurs consécutives avant fermeture du port
n = 0

while(n < 10):
    valueString = ser.readline() # Lecture d'une valeur brute
    value = int(valueString.strip()) # Suppresion des CR/LF
    value = 5.0/1024.0*value # Conversion de la valeur brute en volt
    value = (value - 0.75)/0.01 + 25 # Conversion des volts en °C
    print(value) # Affichage de la valeur
    n = n + 1 # Comptage du nombre de valeurs affichées
    
ser.close() # Fermeture du port série

21.2890625
20.80078125
21.2890625
20.80078125
21.2890625
21.2890625
21.2890625
21.2890625
21.2890625
21.2890625
